In [5]:
import pickle
import pandas
import numpy as np
from datetime import datetime

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback

with open('BTC-USD-60.pkl', 'rb') as f:
    data = pickle.load(f)

#:chunk = filter_df(df_chunk, event_type='Fill')
data = data.sort_values(by=['time'])



In [6]:
## This block process the date/time info, currently not used.
time = np.array(data.time)
#print(time)
new_time = []
year,month,day,hour,minute,second = [],[],[],[],[],[]
for i in range(len(time)):
    new_time.append(datetime.fromtimestamp(time[i]).strftime('%Y-%m-%d %H:%M:%S'))
    year.append(int(new_time[i][:4]))
    month.append(int(new_time[i][5:7]))
    day.append(int(new_time[i][8:10]))
    hour.append(int(new_time[i][11:13]))
    minute.append(int(new_time[i][14:16]))
    #second.append(int(new_time[i][17:19]))
    
print(year[3],month[33],day[333],hour[3333],minute[10000])


2016 1 1 13 51


In [7]:
#print(data)
#data.drop([columns = 'time'])
data = data.drop('time', 1)
data.low = np.log(data.low)
data.high = np.log(data.high)
data.open = np.log(data.open)
data.close = np.log(data.close)

print(data)

          low      high      open     close     volume
280  6.064599  6.064692  6.064599  6.064692   0.072700
279  6.064669  6.064715  6.064669  6.064715   0.947800
278  6.064715  6.064715  6.064715  6.064715   1.633400
277  6.064622  6.064692  6.064692  6.064622  12.566300
276  6.064692  6.064692  6.064692  6.064692   1.953000
275  6.064715  6.066015  6.064715  6.066015  12.114161
274  6.066015  6.066526  6.066015  6.066526  10.660594
273  6.066502  6.068009  6.066502  6.068009   2.610375
272  6.067314  6.067939  6.067430  6.067314   3.142300
271  6.067407  6.067476  6.067430  6.067476   1.973100
270  6.067453  6.067453  6.067453  6.067453   0.750000
269  6.067430  6.067430  6.067430  6.067430   3.791700
268  6.067430  6.067430  6.067430  6.067430   3.441600
267  6.066549  6.067383  6.067383  6.066549   0.896900
266  6.066642  6.066642  6.066642  6.066642   0.046800
265  6.066642  6.066989  6.066642  6.066873   7.627500
264  6.066897  6.067036  6.066897  6.067036   0.170300
263  6.067

In [9]:
mean = data.mean(axis = 0)
std = data.std(axis = 0)
#print(mean)
#print(std)
data = (data - mean)/std
#print(data)

year = np.array(year)
month = np.array(month)
day = np.array(day)
hour = np.array(hour)
minute = np.array(minute)
#second = np.array(second)

year = (year - year.mean())/year.std()
month = (month - month.mean())/month.std()
day = (day - day.mean())/day.std()
hour = (hour - hour.mean())/hour.std()
minute = (minute - minute.mean())/minute.std()
#second = (second - second.mean())/second.std()
#print(data)

In [10]:
def baseline_preprocess(data):
    A = np.array(data.open)
    B = np.array(data.close)
    C = (A + B) / 2

    i = 0
    train_X = []
    train_Y = []
    for i in range(0,len(C) - 60):
        temp = C[i:i+60]
        #print(temp[60])
        train_X.append(temp)
        train_Y.append(C[i + 60])
    #train_X = np.transpose(train_X)
    
    return train_X, train_Y

In [11]:
train_X, train_Y = baseline_preprocess(data)
np.savetxt("train_X_normalized_baseline.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_baseline.csv", train_Y, delimiter = ",")

In [12]:
def fc_preprocess(data, year, hour):
    A = np.array(data.open)
    B = np.array(data.close)
    C = (A + B) / 2

    i = 0
    train_X = []
    train_Y = []
    for i in range(0,len(C) - 60):
        temp = C[i:i+60]
#         temp_year = year[i:i+60]
#         temp_hour = hour[i:i+60]
        #print(temp[60])
        train_X.append()
        train_Y.append(C[i + 60])
    #train_X = np.transpose(train_X)
    
    return train_X, train_Y

In [13]:
train_X, train_Y = fc_preprocess(data, year, hour)
#train_Y = np.log(train_Y)
np.savetxt("train_X_normalized_fc.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_fc.csv", train_Y, delimiter = ",")

TypeError: append() takes exactly one argument (0 given)

In [ ]:
#data = data.drop(columns = ['time'])
def lstm_preprocess(data, year, hour):
    A = np.array(data.low)
    B = np.array(data.high)
    C = np.array(data.open)
    D = np.array(data.close)
    E = np.array(data.volume)
    i = 0
    train_X = []
    train_Y = []
    for i in range(0,len(C) - 60):
        temp = []
        temp.append(A[i:i+60])
        temp.append(B[i:i+60])
        temp.append(C[i:i+60])
        temp.append(D[i:i+60])
        temp.append(E[i:i+60])
#         temp.append(year[i:i+60])
#         temp.append(hour[i:i+60])
        #print(temp[60])
        train_X.append(temp)
        train_Y.append(D[i + 60])
    #print(np.shape(train_X))

    return train_X, train_Y

In [ ]:
train_X, train_Y = lstm_preprocess(data, year, hour)
train_X = np.transpose(train_X, (0,2,1))
#train_Y = np.log(train_Y)
print(train_X.shape)

In [ ]:
np.save("train_X_lstm_normalized_lstm.pkl", train_X)
np.save("train_Y_lstm_normalized_lstm.pkl", train_Y)

In [ ]:
train_Y